In [1]:

import os
import nibabel as nib
import numpy as np

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".nii") or filename.endswith(".nii.gz"): # Check for specific image file extensions
            img_path = os.path.join(folder_path, filename)
            img = nib.load(img_path)
            img_data = img.get_fdata()
            print(f"Shape of image {filename}: {img_data.shape}")
            images.append(img_data)
    return np.array(images)

# Example usage:
path = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'
folder_path = path # Replace with the path to your folder
loaded_images = load_images_from_folder(folder_path)

Shape of image sub-A00000541_ses-20100101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000456_ses-20090101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000838_ses-20100101_acq-mprage_run-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000865_ses-20100101_acq-mprage_echo-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.nii: (192, 256, 256)


In [2]:
import subprocess
import time
import glob
from joblib import Parallel, delayed


directory = '/mnt/md0/cads-phd/For_SPR/mini_data' 

# Running FSL ANAT and getting FSL ANAT Folders 

# n_jobs  = -1 # Number of jobs to run in parallel. -1 means use all available processors.
n_jobs = 10

def process_anat_volumes(directory, n_jobs=6):
    """
    Process anatomical volumes for files in a specified image using parallel computing.

    Parameters:
    directory (str): The directory path where the files are located.
    n_jobs (int): The number of jobs to run in parallel (default is 6).

    Returns:
    None

    Example:
    process_anat_volumes('/path/to/your/directory/', n_jobs=8)
    """
    def anat_volumes(filename):
        full_path = os.path.join(directory, filename)
        return subprocess.run(["fsl_anat", "-i", full_path], capture_output=True)

    Parallel(n_jobs=n_jobs)(delayed(anat_volumes)(filename) for filename in os.listdir(directory))

process_anat_volumes(directory, n_jobs=n_jobs)


In [ ]:
def invwarp(directory):
    return subprocess.run(["invwarp", "--ref="+directory+"/T1.nii.gz", "--warp="+directory+"/T1_to_MNI_nonlin_field.nii.gz", "--out="+directory+"/Warps/"+"std_subject_space", "--verbose"], capture_output=True)


def applywarp_cort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)

def applywarp_subcort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-sub-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"subcort_HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)


def apply_warps(path):
    
    folder = os.path.join(path, "Warps") 
    print(path)
    # if not os.path.exists(folder):
    #     os.mkdir(folder)

    invwarp(path)
    print(path + ' invwarp done')
    time.sleep(3)
    applywarp_cort(path)
    print(path + ' cort done')
    applywarp_subcort(path)
    print(path + ' subcort done')

apply_warps(os.path.join(directory))

#     from joblib import Parallel, delayed
# Parallel(n_jobs=10)(delayed(apply_warps)(file) for file in glob.glob(os.path.join(directory, '*.anat')))



In [ ]:
for i in directory:
    print(i)


In [2]:
import os 
import subprocess


directory =  "/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again"



for filename in os.listdir(directory):
    if filename.endswith(".anat"):

        #  Cortical FSL Stats Volume Extraction
        print(filename+"/Warps/HO_in_subj_t1_space.nii.gz")
        print(filename+"/T1.nii.gz")
        # out_cort = sout = subprocess.Popen(["fslstats", "-K", "./"+filename+"/Warps/HO_in_subj_t1_space.nii.gz", "./"+filename+"/T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        print("Cortical FSL Stats Volume Extraction")
        out_cort = sout = subprocess.Popen(["fslstats", "-K",directory+"/"+filename+"/Warps/HO_in_subj_t1_space.nii.gz", directory+"/"+filename+"/T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

        cort_stdout,stderr = out_cort.communicate()
        print(cort_stdout)
        print("NEW***************************************************************************************")

        # Subcortical FSL Stats Volume Extraction
        print(filename+"/Warps/subcort_HO_in_subj_t1_space.nii.gz")
        print(filename+"/T1.nii.gz")
        print("Subcortical FSL Stats Volume Extraction")
        out_subcort = sout = subprocess.Popen(["fslstats", "-K", directory+"/"+filename+"/Warps/subcort_HO_in_subj_t1_space.nii.gz", directory+"/"+filename+"/T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        subcort_stdout,stderr = out_subcort.communicate()
        print(subcort_stdout)
        print("NEW***************************************************************************************")

sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.anat/Warps/HO_in_subj_t1_space.nii.gz
sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.anat/T1.nii.gz
Cortical FSL Stats Volume Extraction
b'66547 66546.670780 \n7923 7922.960803 \n9000 8999.955475 \n11364 11363.943780 \n1713 1712.991525 \n3463 3462.982868 \n13793 13792.931763 \n18981 18980.906097 \n598 597.997042 \n4517 4516.977654 \n1329 1328.993425 \n4473 4472.977871 \n4182 4181.979311 \n1369 1368.993227 \n2490 2489.987682 \n3018 3017.985069 \n12625 12624.937542 \n5685 5684.971875 \n3518 3517.982596 \n2818 2817.986059 \n5530 5529.972642 \n37560 37559.814184 \n11491 11490.943152 \n2566 2565.987306 \n2848 2847.985910 \n6331 6330.968679 \n6762 6761.966547 \n13346 13345.933975 \n10235 10234.949366 \n8551 8550.957697 \n22142 22141.890459 \n1866 1865.990769 \n13134 13133.935024 \n3899 3898.980711 \n1873 1872.990734 \n9964 9963.950706 \n1145 1144.994335 \n4487 4486.977802 \n4550 4549.977490 \n4415 4414.978158 \n1524 1523.992460 \n7060 7059

In [ ]:
def conv_df_subcort(stdout, atlas, label, region, path):
    HO_cort = pd.read_csv(atlas)
    df_cort = stdout.decode("utf-8")
    df_cort = pd.DataFrame(df_cort.replace('\n',',').split(","), columns=['vol'])
    df_cort['vol']= df_cort['vol'].str.replace(',','', regex=True)
    df_cort[['Voxel', 'Volume']] = df_cort['vol'].str.split(' ', 1, expand=True)
    df_cort = df_cort.drop(columns='vol')
    df_cort['Region'] = region
    df_cort['Target'] = label
    base, patient = os.path.split(os.path.basename(path))
    patient = patient[:-5]
    df_cort['Patient'] = patient
    subcortical_df = HO_cort.merge(df_cort, left_index=True, right_index=True)

    return subcortical_df

In [ ]:
def conv_df_cort(stdout, atlas, label, region, path):
    HO_cort = pd.read_csv(atlas)
    df_cort = stdout.decode("utf-8")
    df_cort = pd.DataFrame(df_cort.replace('\n',',').split(","), columns=['vol'])
    df_cort['vol']= df_cort['vol'].str.replace(',','', regex=True)
    df_cort[['Voxel', 'Volume']] = df_cort['vol'].str.split(' ', 1, expand=True)
    df_cort = df_cort.drop(columns='vol')
    df_cort['Region'] = region
    df_cort['Target'] = label
#     df_cort = df_cort.drop(index=48)
    base, patient = os.path.split(os.path.basename(path))
    patient = patient[:-5]
    df_cort['Patient'] = patient
    cortical_df = HO_cort.merge(df_cort, left_index=True, right_index=True)

    return cortical_df

In [ ]:
def conv_df_subcort(stdout, atlas, label, region, path):
    HO_cort = pd.read_csv(atlas)
    df_cort = stdout.decode("utf-8")
    df_cort = pd.DataFrame(df_cort.replace('\n',',').split(","), columns=['vol'])
    df_cort['vol']= df_cort['vol'].str.replace(',','', regex=True)
    df_cort[['Voxel', 'Volume']] = df_cort['vol'].str.split(' ', 1, expand=True)
    df_cort = df_cort.drop(columns='vol')
    df_cort['Region'] = region
    df_cort['Target'] = label
    base, patient = os.path.split(os.path.basename(path))
    patient = patient[:-5]
    df_cort['Patient'] = patient
    subcortical_df = HO_cort.merge(df_cort, left_index=True, right_index=True)

    return subcortical_df

In [5]:
import os 
import subprocess


directory =  "/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again"



for filename in os.listdir(directory):
    if filename.endswith(".anat"):

        # Subcortical FSL Stats Volume Extraction
        print(filename+"/Warps/subcort_HO_in_subj_t1_space.nii.gz")
        print(filename+"/T1.nii.gz")
        print("Subcortical FSL Stats Volume Extraction")
        out_subcort = sout = subprocess.Popen(["fslstats", "-K", directory+"/"+filename+"/Warps/subcort_HO_in_subj_t1_space.nii.gz", directory+"/"+filename+"/T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        subcort_stdout,stderr = out_subcort.communicate()
        print(subcort_stdout)
        print("NEW***************************************************************************************")

sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.anat/Warps/subcort_HO_in_subj_t1_space.nii.gz
sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.anat/T1.nii.gz
Subcortical FSL Stats Volume Extraction
b'193066 193065.044867 \n394250 394248.049572 \n5723 5722.971687 \n7478 7477.963005 \n2915 2914.985579 \n4709 4708.976704 \n1797 1796.991110 \n23082 23081.885809 \n3352 3351.983417 \n1652 1651.991827 \n531 530.997373 \n199392 199391.013571 \n398207 398205.029996 \n5290 5289.973829 \n7474 7473.963025 \n2818 2817.986059 \n4949 4948.975516 \n1893 1892.990635 \n3714 3713.981626 \n1999 1998.990111 \n452 451.997764 \n'
NEW***************************************************************************************
sub-A00000838_ses-20100101_acq-mprage_run-01_T1w.anat/Warps/subcort_HO_in_subj_t1_space.nii.gz
sub-A00000838_ses-20100101_acq-mprage_run-01_T1w.anat/T1.nii.gz
Subcortical FSL Stats Volume Extraction
b'186344 186344.599776 \n404816 404817.302961 \n5840 5840.018797 \n7807 7807.025128 \n2969 2

In [ ]:
<label index="0" x="58" y="37" z="50">Left Cerebral White Matter</label>

<label index="1" x="70" y="63" z="35">Left Cerebral Cortex </label>

<label index="2" x="57" y="40" z="41">Left Lateral Ventricle</label>

<label index="3" x="51" y="51" z="39">Left Thalamus</label>

<label index="4" x="51" y="71" z="38">Left Caudate</label>

<label index="5" x="56" y="67" z="34">Left Putamen</label>

<label index="6" x="54" y="62" z="35">Left Pallidum</label>

<label index="7" x="44" y="49" z="18">Brain-Stem</label>

<label index="8" x="59" y="54" z="27">Left Hippocampus</label>

<label index="9" x="57" y="61" z="27">Left Amygdala</label>

<label index="10" x="50" y="70" z="33">Left Accumbens</label>

<label index="11" x="29" y="38" z="51">Right Cerebral White Matter</label>

<label index="12" x="25" y="42" z="61">Right Cerebral Cortex </label>

<label index="13" x="35" y="45" z="44">Right Lateral Ventricle</label>

<label index="14" x="38" y="51" z="39">Right Thalamus</label>

<label index="15" x="39" y="72" z="37">Right Caudate</label>

<label index="16" x="34" y="68" z="34">Right Putamen</label>

<label index="17" x="35" y="61" z="35">Right Pallidum</label>

<label index="18" x="31" y="57" z="25">Right Hippocampus</label>

<label index="19" x="32" y="63" z="25">Right Amygdala</label>

<label index="20" x="40" y="69" z="32">Right Accumbens</label>